## Elastic Search
* Instalación y Habilitación
* Conceptos Base
* Indexación
* Monitoreo

### Instalación

En sistemas Unix se procede a ejecutar los siguientes comandos:

In [1]:
!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.5.1-amd64.deb

--2020-01-09 11:36:53--  https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.5.1-amd64.deb
Resolving artifacts.elastic.co (artifacts.elastic.co)... 2a04:4e42:1::734, 151.101.6.222
Connecting to artifacts.elastic.co (artifacts.elastic.co)|2a04:4e42:1::734|:443... failed: Connection timed out.
Connecting to artifacts.elastic.co (artifacts.elastic.co)|151.101.6.222|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 290187264 (277M) [application/octet-stream]
Saving to: ‘elasticsearch-7.5.1-amd64.deb’

elasticsearch-7.5.1 100%[===================>] 276.74M  1.34MB/s    in 4m 27s  

2020-01-09 11:43:30 (1.04 MB/s) - ‘elasticsearch-7.5.1-amd64.deb’ saved [290187264/290187264]



In [ ]:
!sudo dpkg -i elasticsearch-7.5.1-amd64.deb

### Habilitación del servicio

El servicio puede iniciarse con el comando:

In [4]:
!systemctl start elasticsearch.service

Verificando que el servicio este activo

In [64]:
!curl -X GET "localhost:9200/"

{
  "name" : "lasinac",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "f_n46eaUSW2x1p-135n1iw",
  "version" : {
    "number" : "7.5.1",
    "build_flavor" : "default",
    "build_type" : "deb",
    "build_hash" : "3ae9ac9a93c95bd0cdc054951cf95d88e1e18d96",
    "build_date" : "2019-12-16T22:57:37.835892Z",
    "build_snapshot" : false,
    "lucene_version" : "8.3.0",
    "minimum_wire_compatibility_version" : "6.8.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


In [65]:
!curl -X GET "localhost:9200/_cat/health?v"

epoch      timestamp cluster       status node.total node.data shards pri relo init unassign pending_tasks max_task_wait_time active_shards_percent
1578669031 15:10:31  elasticsearch green           1         1      0   0    0    0        0             0                  -                100.0%


In [92]:
!curl -X GET localhost:9200/_cat/indices?v 

health status index uuid                   pri rep docs.count docs.deleted store.size pri.store.size
yellow open   slr   E_AVV-6ETeG0RDUmANjXdQ   1   1          1            0     14.2kb         14.2kb


In [72]:
!curl -X GET "localhost:9200/_search"

{"took":0,"timed_out":false,"_shards":{"total":0,"successful":0,"skipped":0,"failed":0},"hits":{"total":{"value":0,"relation":"eq"},"max_score":0.0,"hits":[]}}

### Configurando Elasticsearch

* Elasticsearch data se guarda en: /var/lib/elasticsearch 
* Archivo de configuraciòn en: /etc/elasticsearch 
* Opciones adicionales de inicio de java configurables desde archivo: /etc/default/elasticsearch file.

## Conceptos Básicos

* __Documento:__ Unidad básica de informacón indexable, recuperable. (**json**)
* __Índice:__ Conjunto de documentos con características similares. 
* __Nodo:__ Instancia de Elastic Search; Almacena todos los documentos.
* __Cluster:__ Conjunto de uno o más nodos. Pueden existir diferentes índices dentro de un clúster.
* __Shard:__ Fragmento de un índice que contiene parte de los documentos.
* __Réplica:__ Copia de un shard primario
* __Settings:__ Configuración del índice: # de réplicas, # de shards a nivel de cluster. Diferentes indices pueden tener diferentes valores, mientras que todos los nodos del cluster tendrán los mismos valores.
* __Mappings:__ Modelo de datos de un índice. Campos y Tipos de dato.

### Concepto de Indexación Inversa
<img src="images/inverse_indexing.png">

### Indexando un documento json

In [102]:
!curl -XDELETE localhost:9200/*

{"acknowledged":true}

In [103]:
!curl -XPUT http://localhost:9200/slr/books/1 \
        -H 'Content-Type: application/json'\
        -d '{"author": "Cohen, Paul R", "publisher": "MIT press Cambridge, MA", "title": "Empirical methods for artificial intelligence", "volume": 139, "year": 1995}'

{"_index":"slr","_type":"books","_id":"1","_version":1,"result":"created","_shards":{"total":2,"successful":1,"failed":0},"_seq_no":0,"_primary_term":1}

In [106]:
!curl -XPUT http://localhost:9200/slr/books/2 \
        -H 'Content-Type: application/json' \
        -d '{"title": "hello world", "content":"world at war", "tags":["one", "two", "three"]}'

{"_index":"slr","_type":"books","_id":"2","_version":1,"result":"created","_shards":{"total":2,"successful":1,"failed":0},"_seq_no":1,"_primary_term":1}

Creación automática de identificador

In [109]:
!curl -XPOST http://localhost:9200/slr/books \
        -H 'Content-Type: application/json' \
        -d @elastic_search_books.json

{"_index":"slr","_type":"books","_id":"haAPkG8BngSePPhFJPyW","_version":1,"result":"created","_shards":{"total":2,"successful":1,"failed":0},"_seq_no":2,"_primary_term":1}

### Recuperando Documentos

In [110]:
!curl -XGET http://localhost:9200/_search

{"took":1,"timed_out":false,"_shards":{"total":1,"successful":1,"skipped":0,"failed":0},"hits":{"total":{"value":3,"relation":"eq"},"max_score":1.0,"hits":[{"_index":"slr","_type":"books","_id":"1","_score":1.0,"_source":{"author": "Cohen, Paul R", "publisher": "MIT press Cambridge, MA", "title": "Empirical methods for artificial intelligence", "volume": 139, "year": 1995}},{"_index":"slr","_type":"books","_id":"2","_score":1.0,"_source":{"title": "hello world", "content":"world at war", "tags":["one", "two", "three"]}},{"_index":"slr","_type":"books","_id":"haAPkG8BngSePPhFJPyW","_score":1.0,"_source":{	"book1525739": {		"title": "Elasticsearch Indexing: Improve search experiences with ElasticSearch’s powerful indexing functionality – learn how with this practical ElasticSearch tutorial, packed with tips!",		"author": "Huseyin Akdogan",		"publisher": "Packt Publishing",		"isbn": "978-1-78398-702-3",		"year": "2015",		"series": "",		"edition": "",		"volume": ""	},	"book1525753": {		"ti

In [112]:
!curl -XGET http://localhost:9200/slr/books/1

{"_index":"slr","_type":"books","_id":"1","_version":1,"_seq_no":0,"_primary_term":1,"found":true,"_source":{"author": "Cohen, Paul R", "publisher": "MIT press Cambridge, MA", "title": "Empirical methods for artificial intelligence", "volume": 139, "year": 1995}}

In [113]:
!curl -XGET http://localhost:9200/slr/books/haAPkG8BngSePPhFJPyW

{"_index":"slr","_type":"books","_id":"haAPkG8BngSePPhFJPyW","_version":1,"_seq_no":2,"_primary_term":1,"found":true,"_source":{	"book1525739": {		"title": "Elasticsearch Indexing: Improve search experiences with ElasticSearch’s powerful indexing functionality – learn how with this practical ElasticSearch tutorial, packed with tips!",		"author": "Huseyin Akdogan",		"publisher": "Packt Publishing",		"isbn": "978-1-78398-702-3",		"year": "2015",		"series": "",		"edition": "",		"volume": ""	},	"book1525753": {		"title": "Elasticsearch Server, 2nd Edition: A practical guide to building fast, scalable, and flexible search solutions with clear and easy-to-understand examples",		"author": "Marek Rogozinski, Rafal Kuc",		"publisher": "Packt Publishing",		"isbn": "978-1-78398-052-9",		"year": "2014",		"series": "",		"edition": "",		"volume": ""	}}}

### Shards and Replicas

* Un indice esta compuesto de una o mas **shards**.
* Cada **shard** contiene parte del conjunto de docuemntos
* Cada **shard** puede tener **replicas**
* Valores globales están definidos en el archivo elasticsearch.yml: 5 shards, 1 replica

### Shards y Replicas
* Más shards permiten extender los índices a mas servidores (i.e. manejar más documentos)
* Más shards implica menos recursos para buscar (_fetch_) un documento particular
* Más shards implica problemas al buscar entre índices: fase de agregación 
* Más réplicas implica mayor tolerancia a fallas
* Más réplicas implica mayor rendimiento de la consulta.

Las réplicas permiten una tolerancia a fallas ya que si el shard no es encontrado o no existe la copia se puede utilizar, de ahi que a mas copias se puede garantizar que no haya perdida datos

### Creando un índice
Crear un indice de nombre **repo_tesis** con parámetros por defecto


In [115]:
!curl -XPUT http://localhost:9200/repo_tesis/

{"acknowledged":true,"shards_acknowledged":true,"index":"repo_tesis"}

### Alterando archivo de configuracion yml
A veces la creación automática de índices no es adecuada: un typo en un nombre de índice puede destruir horas de procesamiento de script. Para controlar esta situación se puede desactivar o puntualizar el comportamineto de la creación de índices por elastic search editando el archivo **elasticsearch.yml**.

Desactivar indexación automática:

    action.auto_create_index: false

Permitir indexación automática para índices que inician con la letra **a**, desactivarla para **an**, y que cualquier otro indice sea creado manualmente (-*). Observar el orden.

    action.auto_creat_index: -an*, +a*, -*
    
Para acceder:

    sudo -i
    nano /etc/elasticsearch/elasticsearch.yml

### Configurando Manualmente un índice:

Permite configurar de manera manual el número de shards y réplicas.

In [127]:
!curl -XPUT http://localhost:9200/repo_tesis_2/ -d '{"settings":{"number_of_shards":1, "number_of_replicas": 2}}' \
                                                -H 'Content-Type: application/json'

{"acknowledged":true,"shards_acknowledged":true,"index":"repo_tesis_2"}

# ¡Falta la estructura o mapeo de los datos!

Dado que no se ha proveido del mapping, es necesario eliminar el indice antes creado para configurar


### Mapping Data